In [ ]:
import pynetlogo
import ipywidgets as widgets
import matplotlib.pyplot as plt
import random
from ambiente import EnvironmentModel

# Inicializa NetLogo
netlogo = pynetlogo.NetLogoLink(gui=False, netlogo_home=r'C:\Program Files\NetLogo 6.4.0')
netlogo.load_model(r'C:\Users\david\Desktop\Bolsa_Investigacao\projeto\simulacao_fogo.nlogo')

model = EnvironmentModel(5, 10, 10)
netlogo.command('setup')

# 4) WIDGETS PARA CRIAR A INTERFACE INTERATIVA NO JUPYTER
################################################################################

# Slider para definir o número de iterações
iterations_slider = widgets.IntSlider(
    value=200, 
    min=10, 
    max=500, 
    step=10,
    description='Iterações:',
    continuous_update=False
)

# Botão para iniciar a simulação
run_button = widgets.Button(
    description='Iniciar Simulação', 
    button_style='success',
    tooltip='Clique para iniciar a simulação'
)

# Botão para apagar fogo manualmente
stop_fire_button = widgets.Button(
    description='Apagar Fogo',
    button_style='warning',
    tooltip='Clique para apagar o fogo imediatamente'
)

# Área de saída para logs e gráficos
output_area = widgets.Output()

# Listas para armazenar os valores coletados a cada tick
burned_area_evol = []
forested_area_evol = []
timesteps = []

def run_simulation(b):
    """Função chamada ao clicar no botão 'Iniciar Simulação'."""
    with output_area:
        clear_output()
        print("Preparando simulação...")
        
        # Reinicia o modelo NetLogo para começar do zero e zera contadores
        netlogo.command('setup')
        model.fires_created = 0
        model.fires_detected = 0

        # Limpa dados antigos
        burned_area_evol.clear()
        forested_area_evol.clear()
        timesteps.clear()

        iterations = iterations_slider.value
        for i in range(iterations):
            # Com probabilidade de 30%, iniciamos fogo (isso incrementa fires_created)
            if random.random() < 0.3:
                model.start_fire()

            # Executa um passo (1 tick) do nosso modelo
            model.step()

            # Coleta dados do NetLogo:
            #  - burned-trees = quantas árvores foram queimadas
            #  - count patches with [pcolor = green] = quantas ainda estão verdes
            burned_trees = netlogo.report("burned-trees")
            forested_trees = netlogo.report("count patches with [pcolor = green]")

            burned_area_evol.append(burned_trees)
            forested_area_evol.append(forested_trees)
            timesteps.append(i)

            print(f"Iteração {i} | Queimado: {burned_trees}, Florestado: {forested_trees}")

        # Ao final, mostra quantos incêndios foram criados e detectados
        print("\nSimulação finalizada!")
        print(f"Número de incêndios criados: {model.fires_created}")
        print(f"Número de incêndios detectados: {model.fires_detected}")

        # Gera um gráfico com matplotlib
        plt.figure(figsize=(6, 4))
        plt.plot(timesteps, burned_area_evol, label='Árvores Queimadas', color='red')
        plt.plot(timesteps, forested_area_evol, label='Árvores Florestadas', color='green')
        plt.xlabel('Iterações')
        plt.ylabel('Quantidade')
        plt.title('Evolução de Árvores Queimadas x Florestadas')
        plt.legend()
        plt.grid(True)
        plt.show()

def stop_fire_now(b):
    """Função chamada ao clicar no botão 'Apagar Fogo'."""
    with output_area:
        clear_output(wait=True)
        model.stop_fire()
        print("Fogo apagado manualmente!")

# Associa as funções aos botões
run_button.on_click(run_simulation)
stop_fire_button.on_click(stop_fire_now)

# Exibe os widgets na tela (Jupyter)
display(
    widgets.VBox([
        iterations_slider,
        widgets.HBox([run_button, stop_fire_button]),
        output_area
    ])
)

TypeError: object.__init__() takes exactly one argument (the instance to initialize)